In [2]:
# Import Libraries

import json
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Data Preprocessing

- **Data Cleaning:** Dealing with `null` values
- **Categorical Columns Processing:** Converting all the non-numerical(categorical, dates) to numerical values

In [3]:
# TODO: Import from utils
def define_user_profile(loan_status):
    # Define user profile based on loan status
    if loan_status in ('Fully Paid', 'Current'):
        return 'Payer'
    elif loan_status == loan_status:
        return 'Defaulter'
    else:
        return np.nan

In [4]:
df = pd.read_csv('../data/accepted_2007_to_2018Q4.csv')
df['user_profile']  = df['loan_status'].apply(define_user_profile)
df.drop(['loan_status'], axis=1, inplace=True)
df.head()

/var/folders/3z/1gg3czrd2ps4jtgnrk_j_nqmgw5xwd/T/ipykernel_76559/2454511268.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/accepted_2007_to_2018Q4.csv')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,user_profile
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,Payer


In [5]:
# Columns selected from EDA
selected_cols = ['loan_amnt', 'int_rate', 'installment', 'issue_d', 'grade', 'sub_grade', 
                 'purpose', 'debt_settlement_flag', 'earliest_cr_line',
                 'home_ownership', 'fico_range_low', 'fico_range_high', 'dti']

df_selected = df[selected_cols + ['user_profile']]
df_selected.head()

,loan_amnt,int_rate,installment,issue_d,grade,sub_grade,purpose,debt_settlement_flag,earliest_cr_line,home_ownership,fico_range_low,fico_range_high,dti,user_profile
0,3600.0,13.99,123.03,Dec-2015,C,C4,debt_consolidation,N,Aug-2003,MORTGAGE,675.0,679.0,5.91,Payer
1,24700.0,11.99,820.28,Dec-2015,C,C1,small_business,N,Dec-1999,MORTGAGE,715.0,719.0,16.06,Payer
2,20000.0,10.78,432.66,Dec-2015,B,B4,home_improvement,N,Aug-2000,MORTGAGE,695.0,699.0,10.78,Payer
3,35000.0,14.85,829.90,Dec-2015,C,C5,debt_consolidation,N,Sep-2008,MORTGAGE,785.0,789.0,17.06,Payer
4,10400.0,22.45,289.91,Dec-2015,F,F1,major_purchase,N,Jun-1998,MORTGAGE,695.0,699.0,25.37,Payer


### Data Cleaning

In [6]:
def null_df(df):
    null_percentage = (df.isnull().sum() / len(df)) * 100

    # Convert to DataFrame for better visualization
    null_percentage_df = pd.DataFrame({"Column": null_percentage.index, "Null_Percentage": null_percentage.values})
    null_percentage_df = null_percentage_df[null_percentage_df['Null_Percentage'] > 0.0]

    return null_percentage_df

In [7]:
null_df(df_selected)

,Column,Null_Percentage
0,loan_amnt,0.001460
1,int_rate,0.001460
2,installment,0.001460
3,issue_d,0.001460
4,grade,0.001460
5,sub_grade,0.001460
6,purpose,0.001460
7,debt_settlement_flag,0.001460
8,earliest_cr_line,0.002743
9,home_ownership,0.001460


In [8]:
# Drop all rows with null values in `USER PROFILE`
df_selected = df_selected[df_selected['user_profile'].notnull()]
null_df(df_selected)

,Column,Null_Percentage
8,earliest_cr_line,0.001283
12,dti,0.075686


In [9]:
# Drop all rows with null values in earliest_cr_line because its a `date` column and its not possible to fill it with any value
df_selected = df_selected[df_selected['earliest_cr_line'].notnull()]
# DTI is a continuous variable and can be filled with mean value
df_selected['dti'] = df_selected['dti'].fillna(df_selected['dti'].mean())
null_df(df_selected)

,Column,Null_Percentage


### Data Processing/Feature Engg

#### Date columns

In [10]:
earliest_issue_d = min(pd.to_datetime(df['issue_d']))
print("Earliest Issue of Loan: {}".format(earliest_issue_d))

# Calculate the number of months since the earliest issue date and assign it to the issue_d column
df_selected['issue_d'] = pd.to_datetime(df_selected['issue_d'])
df_selected['issue_d'] = (df_selected['issue_d'] - earliest_issue_d) / (np.timedelta64(1, 'D') * 30)
df_selected['issue_d'].head()

/var/folders/3z/1gg3czrd2ps4jtgnrk_j_nqmgw5xwd/T/ipykernel_76559/913015718.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  earliest_issue_d = min(pd.to_datetime(df['issue_d']))


Earliest Issue of Loan: 2007-06-01 00:00:00


/var/folders/3z/1gg3czrd2ps4jtgnrk_j_nqmgw5xwd/T/ipykernel_76559/913015718.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_selected['issue_d'] = pd.to_datetime(df_selected['issue_d'])


0    103.5
1    103.5
2    103.5
3    103.5
4    103.5
Name: issue_d, dtype: float64

In [11]:
min_earliest_credit_line = min(pd.to_datetime(df['earliest_cr_line']))
print("Min Earliest Credit Line: {}".format(min_earliest_credit_line))

# Problem with considering min_earliest_credit_line is that it is in the future, a person with an even earlier credit line will have a negative value
# To deal with this and to be on safe side, we will assign min_earliest_credit_line to be 1900-01-01
min_earliest_credit_line = pd.to_datetime('1900-01-01')
df_selected['earliest_cr_line'] = pd.to_datetime(df_selected['earliest_cr_line'])
df_selected['earliest_cr_line'] = (df_selected['earliest_cr_line'] - min_earliest_credit_line) / (np.timedelta64(1, 'D') * 30)
df_selected['earliest_cr_line'].head()

/var/folders/3z/1gg3czrd2ps4jtgnrk_j_nqmgw5xwd/T/ipykernel_76559/3477225754.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  min_earliest_credit_line = min(pd.to_datetime(df['earliest_cr_line']))


Min Earliest Credit Line: 1933-03-01 00:00:00


/var/folders/3z/1gg3czrd2ps4jtgnrk_j_nqmgw5xwd/T/ipykernel_76559/3477225754.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_selected['earliest_cr_line'] = pd.to_datetime(df_selected['earliest_cr_line'])


0    1261.066667
1    1216.433333
2    1224.566667
3    1323.000000
4    1198.166667
Name: earliest_cr_line, dtype: float64

#### Dealing with grade/subgrade column

In [12]:
# Extract sub-grade number from sub-grade column
df_selected['sub_grade_no'] = df_selected['sub_grade'].apply(lambda x: int(x.strip()[1:]))

In [13]:
# Convert grade into one_hot encoding columns
grade_cols = ['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G']
df_selected = pd.get_dummies(df_selected, columns=['grade'])
df_selected.head()

,loan_amnt,int_rate,installment,issue_d,sub_grade,purpose,debt_settlement_flag,earliest_cr_line,home_ownership,fico_range_low,...,dti,user_profile,sub_grade_no,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G
0,3600.0,13.99,123.03,103.5,C4,debt_consolidation,N,1261.066667,MORTGAGE,675.0,...,5.91,Payer,4,False,False,True,False,False,False,False
1,24700.0,11.99,820.28,103.5,C1,small_business,N,1216.433333,MORTGAGE,715.0,...,16.06,Payer,1,False,False,True,False,False,False,False
2,20000.0,10.78,432.66,103.5,B4,home_improvement,N,1224.566667,MORTGAGE,695.0,...,10.78,Payer,4,False,True,False,False,False,False,False
3,35000.0,14.85,829.90,103.5,C5,debt_consolidation,N,1323.000000,MORTGAGE,785.0,...,17.06,Payer,5,False,False,True,False,False,False,False
4,10400.0,22.45,289.91,103.5,F1,major_purchase,N,1198.166667,MORTGAGE,695.0,...,25.37,Payer,1,False,False,False,False,False,True,False


In [14]:
# Finalizing the grade columns
for col in grade_cols:
    df_selected[col] = df_selected[col].apply(int) * df_selected['sub_grade_no']
df_selected = df_selected.drop(['sub_grade_no', 'sub_grade'], axis=1)
df_selected.head()

,loan_amnt,int_rate,installment,issue_d,purpose,debt_settlement_flag,earliest_cr_line,home_ownership,fico_range_low,fico_range_high,dti,user_profile,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G
0,3600.0,13.99,123.03,103.5,debt_consolidation,N,1261.066667,MORTGAGE,675.0,679.0,5.91,Payer,0,0,4,0,0,0,0
1,24700.0,11.99,820.28,103.5,small_business,N,1216.433333,MORTGAGE,715.0,719.0,16.06,Payer,0,0,1,0,0,0,0
2,20000.0,10.78,432.66,103.5,home_improvement,N,1224.566667,MORTGAGE,695.0,699.0,10.78,Payer,0,4,0,0,0,0,0
3,35000.0,14.85,829.90,103.5,debt_consolidation,N,1323.000000,MORTGAGE,785.0,789.0,17.06,Payer,0,0,5,0,0,0,0
4,10400.0,22.45,289.91,103.5,major_purchase,N,1198.166667,MORTGAGE,695.0,699.0,25.37,Payer,0,0,0,0,0,1,0


#### Dealing with other categorical columns
- We will process and store in two different ways:
1. Converting columns into `one-hot encoding`
2. Converting columns into categories

In [15]:
df_selected['home_ownership'] = df_selected['home_ownership'].apply(lambda x: x if x in ['MORTGAGE', 'RENT', 'OWN'] else 'OTHER')

In [16]:
# One-hot encoding for categorical columns
cat_cols = ['home_ownership', 'debt_settlement_flag', 'purpose']
df_one_hot_enc = pd.get_dummies(df_selected, columns=['home_ownership', 'debt_settlement_flag', 'purpose'])

# Convert all columns to integer
for col in list(df_one_hot_enc.columns):
    for cat_col in cat_cols:
        if cat_col in col:
            df_one_hot_enc[col] = df_one_hot_enc[col].apply(int)
df_one_hot_enc.head()

,loan_amnt,int_rate,installment,issue_d,earliest_cr_line,fico_range_low,fico_range_high,dti,user_profile,grade_A,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,3600.0,13.99,123.03,103.5,1261.066667,675.0,679.0,5.91,Payer,0,...,0,0,0,0,0,0,0,0,0,0
1,24700.0,11.99,820.28,103.5,1216.433333,715.0,719.0,16.06,Payer,0,...,0,0,0,0,0,0,0,1,0,0
2,20000.0,10.78,432.66,103.5,1224.566667,695.0,699.0,10.78,Payer,0,...,1,0,0,0,0,0,0,0,0,0
3,35000.0,14.85,829.90,103.5,1323.000000,785.0,789.0,17.06,Payer,0,...,0,0,0,0,0,0,0,0,0,0
4,10400.0,22.45,289.91,103.5,1198.166667,695.0,699.0,25.37,Payer,0,...,0,0,1,0,0,0,0,0,0,0


In [17]:
# Standardize the data using Gaussian with zero mean and unit variance
scaler = StandardScaler()
# Columns to standardize
cols = list(df_one_hot_enc.columns)
cols = [ix for ix in cols if ix not in ['user_profile']]
df_one_hot_enc[cols] = scaler.fit_transform(df_one_hot_enc[cols])

# Store the standard scaler artifact
with open('../data/artefacts/' + 'standard_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

df_one_hot_enc.head()

,loan_amnt,int_rate,installment,issue_d,earliest_cr_line,fico_range_low,fico_range_high,dti,user_profile,grade_A,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,-1.245572,0.185657,-1.208137,-0.233874,0.463785,-0.714566,-0.714553,-0.910873,Payer,-0.432394,...,-0.267022,-0.079325,-0.151075,-0.110946,-0.082827,-0.256364,-0.02529,-0.105080,-0.083157,-0.032293
1,1.050349,-0.228237,1.401600,-0.233874,-0.003382,0.497185,0.497166,-0.194974,Payer,-0.432394,...,-0.267022,-0.079325,-0.151075,-0.110946,-0.082827,-0.256364,-0.02529,9.516545,-0.083157,-0.032293
2,0.538935,-0.478644,-0.049223,-0.233874,0.081748,-0.108690,-0.108693,-0.567382,Payer,-0.432394,...,3.745017,-0.079325,-0.151075,-0.110946,-0.082827,-0.256364,-0.02529,-0.105080,-0.083157,-0.032293
3,2.171106,0.363632,1.437607,-0.233874,1.112027,2.617749,2.617674,-0.124442,Payer,-0.432394,...,-0.267022,-0.079325,-0.151075,-0.110946,-0.082827,-0.256364,-0.02529,-0.105080,-0.083157,-0.032293
4,-0.505655,1.936432,-0.583522,-0.233874,-0.194575,-0.108690,-0.108693,0.461678,Payer,-0.432394,...,-0.267022,-0.079325,6.619210,-0.110946,-0.082827,-0.256364,-0.02529,-0.105080,-0.083157,-0.032293


In [18]:
# Label encoding categorical columns into
df_label_enc = df_selected.copy()

encoder = LabelEncoder()
cat_cols = ['home_ownership', 'debt_settlement_flag', 'purpose']
for col in cat_cols:
    df_label_enc[col] = encoder.fit_transform(df_label_enc[col])
    print(col+" dictionary: {}".format(dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))))
    # Store the dictionary for later use
    with open('../data/artefacts/'+col+'_dictionary.json', 'w') as f:
        json.dump(dict(zip(encoder.classes_, [int(ix) for ix in encoder.transform(encoder.classes_)])), f)

df_label_enc.head()

home_ownership dictionary: {'MORTGAGE': np.int64(0), 'OTHER': np.int64(1), 'OWN': np.int64(2), 'RENT': np.int64(3)}
debt_settlement_flag dictionary: {'N': np.int64(0), 'Y': np.int64(1)}
purpose dictionary: {'car': np.int64(0), 'credit_card': np.int64(1), 'debt_consolidation': np.int64(2), 'educational': np.int64(3), 'home_improvement': np.int64(4), 'house': np.int64(5), 'major_purchase': np.int64(6), 'medical': np.int64(7), 'moving': np.int64(8), 'other': np.int64(9), 'renewable_energy': np.int64(10), 'small_business': np.int64(11), 'vacation': np.int64(12), 'wedding': np.int64(13)}


,loan_amnt,int_rate,installment,issue_d,purpose,debt_settlement_flag,earliest_cr_line,home_ownership,fico_range_low,fico_range_high,dti,user_profile,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G
0,3600.0,13.99,123.03,103.5,2,0,1261.066667,0,675.0,679.0,5.91,Payer,0,0,4,0,0,0,0
1,24700.0,11.99,820.28,103.5,11,0,1216.433333,0,715.0,719.0,16.06,Payer,0,0,1,0,0,0,0
2,20000.0,10.78,432.66,103.5,4,0,1224.566667,0,695.0,699.0,10.78,Payer,0,4,0,0,0,0,0
3,35000.0,14.85,829.90,103.5,2,0,1323.000000,0,785.0,789.0,17.06,Payer,0,0,5,0,0,0,0
4,10400.0,22.45,289.91,103.5,6,0,1198.166667,0,695.0,699.0,25.37,Payer,0,0,0,0,0,1,0


In [19]:
# Storing the data

df_one_hot_enc.to_csv('../data/processed/standardized_data.csv', index=False)
df_label_enc.to_csv('../data/processed/raw_data.csv', index=False)